In [2]:
# Código principal, proveniente de ML_28
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from google.cloud import bigquery
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# Inicializar cliente BigQuery
client = bigquery.Client()

# Cargar datos desde BigQuery
query = "SELECT * FROM `pf-henry-404414.data_machine_learning.wb_data_machine_learning_bis`"
data_original = client.query(query).to_dataframe()
data = data_original.copy()

# Preprocesamiento: One-Hot Encoding para 'Pais'
data = pd.get_dummies(data, columns=['Pais'])

# Obtener la lista de todos los países únicos
all_countries = data_original['Pais'].unique()


# Lista de columnas numéricas (sin incluir la variable objetivo)
numeric_columns = [
    'Tasa_Natalidad', 'Emisiones_CO2', 'Educacion_obligatoria_en_anios',
    'gasto_salud_per_capita_ppp', 'Tasa_Mortalidad', 'Gasto_Salud_Gobierno',
    'gasto_salud_gobierno_per_capita_ppp', 'gasto_salud_privado_pct_gasto_salud_actual',
    'logro_educativo_secundaria_inferior_pct_poblacion_25_anios_mas', 'PIB_per_capita',
    'indice_gini', 'gasto_educacion_gobierno_pct_pib', 'Esperanza_vida_femenina',
    'Tasa_alfabetizacion_adultos', 'tasa_mortalidad_lesiones_trafico',
    'mortalidad_por___enfermedades_cardiovasculares_cancer_diabetes_enf_respiratorias_pct',
    'Mortalidad_adulta_femenina', 'Mortalidad_adulta_masculina', 'Mortalidad_infantil',
    'contaminacion_pct_poblacion_excede_oms', 'personas_saneamiento_basico_pct_poblacion',
    'Acceso_agua_potable', 'estabilidad_politica', 'brecha_pobreza_2_15_dolars_a_day',
    'prevalencia_desnutricion_pct_poblacion', 'Poblacion_rural', 'desempleo_total_ilo',
    'poblacion_urbana'
]

# Define la lista de indicadores excluyendo la variable objetivo y asegurándote de que no incluya columnas no numéricas
indicators = [col for col in numeric_columns if col != 'Esperanza_vida_total']


# Separar variables predictoras y objetivo
X = data[numeric_columns]
y = data['Esperanza_vida_total']

# Normalización de variables numéricas
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Aplicación de PCA
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Entrenar y evaluar Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# Entrenar y evaluar Perceptrón Multicapa (Red Neuronal)
nn = MLPRegressor(random_state=42)
nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)

# Entrenar y evaluar Regresión Lineal
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# Comparar métricas
print("Random Forest MSE:", mean_squared_error(y_test, rf_pred))
print("Random Forest R2:", r2_score(y_test, rf_pred))
print("Red Neuronal MSE:", mean_squared_error(y_test, nn_pred))
print("Red Neuronal R2:", r2_score(y_test, nn_pred))
print("Regresión Lineal MSE:", mean_squared_error(y_test, lr_pred))
print("Regresión Lineal R2:", r2_score(y_test, lr_pred))

# Validación cruzada y ajuste de hiperparámetros para Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

# Reentrenamiento con la mejor configuración
rf_optimized = RandomForestRegressor(
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    random_state=42
)
rf_optimized.fit(X_train, y_train)

# Función para extrapolación de indicadores
def extrapolate_indicators(data, country, indicators, target_year):
    extrapolated_values = []
    for indicator in indicators:
        country_data = data[data['Pais'] == country]
        X = country_data['Anio'].values.reshape(-1, 1)
        y = country_data[indicator].values

        model = LinearRegression()
        model.fit(X, y)

        predicted_value = model.predict([[target_year]])[0]
        extrapolated_values.append(predicted_value)

    return np.array(extrapolated_values)

# Función para predecir esperanza de vida
def predict_life_expectancy(country, year, model, data_original, scaler, pca, indicators):
    extrapolated_values = extrapolate_indicators(data_original, country, indicators, year)

    # Aplicar las transformaciones a los valores extrapolados
    scaled_values = scaler.transform([extrapolated_values])
    pca_values = pca.transform(scaled_values)

    # Realizar la predicción
    predicted_life_expectancy = model.predict(pca_values)[0]
    return predicted_life_expectancy

    #¿Qué país tiene la esperanza de vida más alta para el 2040?
#Año de interés
year_of_interest = 2040

# Diccionario para almacenar las predicciones de esperanza de vida para todos los países en 2040
predictions_2040 = {}

# Iterar sobre todos los países y obtener las predicciones
for country in all_countries:
    predicted_life_expectancy = predict_life_expectancy(country, year_of_interest, rf_optimized, data_original, scaler, pca, indicators)
    predictions_2040[country] = predicted_life_expectancy

# Encontrar el país con la esperanza de vida más alta en 2040
country_with_highest_life_expectancy = max(predictions_2040, key=predictions_2040.get)
highest_life_expectancy = predictions_2040[country_with_highest_life_expectancy]

print(f"El país con la esperanza de vida más alta en {year_of_interest} es {country_with_highest_life_expectancy} con una esperanza de vida de {highest_life_expectancy} años.")


Random Forest MSE: 0.5303251167089678
Random Forest R2: 0.9606226069858075
Red Neuronal MSE: 2413.1394881568735
Red Neuronal R2: -178.17903382156325
Regresión Lineal MSE: 0.44343721792979296
Regresión Lineal R2: 0.9670741568570208
Fitting 5 folds for each of 27 candidates, totalling 135 fits
El país con la esperanza de vida más alta en 2040 es Canada con una esperanza de vida de 81.29183414770006 años.
